# PAN datasets 

## Step 1: Extract municipalities and match them with their coordinates in BestuurlijkeGebieden

TO DO - prepare input 
- all PAN datasets + info (pid, dansSpatialCoverageText) 

In [1]:
# Get data from MongoDB database 
from pymongo import MongoClient

# Replace with your MongoDB connection string
CONNECTION_STRING = "mongodb://127.0.0.1:27018"  # For local MongoDB

# Connect to the database and collection
client = MongoClient(CONNECTION_STRING)

db = client["archaeology_metadata"] # Define database
collection = db["collection"] # Define collection


In [2]:
# Find all PAN datasets 
query = {
    "ore:describes.author.citation:authorName": "Portable Antiquities of the Netherlands"}

pan = collection.find(query)


In [8]:
# Only select specific fields



In [ ]:
#### Scrap 


# Somehow select the right level of granularity 
# Maybe by checking if it's a municipality? 
# What if a lower level is present? 

import requests

# Set up your GeoNames username and API URL
username = "your_geonames_username"
base_url = "http://api.geonames.org/searchJSON"

# Function to classify place name
def classify_place(place_name):
    # Prepare the query parameters
    params = {
        'q': place_name,   # The place name to search for
        'maxRows': 1,      # Get the best match
        'username': username
    }
    
    # Send the request to GeoNames API
    response = requests.get(base_url, params=params)
    data = response.json()
    
    # Check if there are results
    if data['geonames']:
        # Extract the first result (best match)
        result = data['geonames'][0]
        
        # Classify the place using 'fcode' and 'adminCode'
        fcode = result.get('fcode')
        country = result.get('countryName')
        admin1 = result.get('adminName1')  # First-level administrative division (state/province)
        admin2 = result.get('adminName2')  # Second-level (county)
        
        # Output the classification based on 'fcode'
        if fcode == 'PPL':
            classification = 'City or Town'
        elif fcode == 'PPLX':
            classification = 'Municipality or Administrative Division'
        elif fcode == 'STREET':
            classification = 'Street Address'
        elif fcode.startswith('ADM'):
            classification = f'Administrative Division (e.g., {admin1})'
        else:
            classification = 'Unknown Type'

        # Return classification along with some key data
        return {
            'place_name': place_name,
            'country': country,
            'admin1': admin1,
            'admin2': admin2,
            'classification': classification
        }
    else:
        return {'place_name': place_name, 'classification': 'Not Found'}

# Example usage
place = "Amsterdam"
classification_result = classify_place(place)
print(classification_result)



## Step 2: Convert to WGS84, display on map

In [34]:
import folium
import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import Proj, transform

# Define the UTM projection
utm_proj = Proj(init='epsg:28992')  # Data in Bestuurlijke Gebieden is EPSG:28992 
wgs_proj = Proj(init='epsg:4326')  # WGS84 (longitude/latitude), goal

# Paste raw coordinates
raw_coordinates = "78201.037 436044.553 78208.877 436052.324 78212.974 436056.385 78218.447 436061.82 78219.551 436062.917 78221.295 436064.648 78239.818 436083.042 78241.584 436084.796 78253.161 436096.292 78253.836 436096.964 78264.291 436107.346 78266.025 436109.067 78296.026 436138.861 78354.715 436195.22 78421.007 436258.381 78456.698 436292.713 78473.557 436308.931 78477.84 436313.052 78538.971 436370.952 78597.737 436427.073 78607.899 436436.733 78608.83 436437.619 78609.282 436438.048 78612.349 436441.156 78612.372 436441.18 78612.838 436441.652 78627.278 436456.285 78630.571 436459.497 78658.83 436486.94 78680.41 436507.907 78685.358 436512.714 78691.845 436519.066 78696.181 436523.312 78792.7 436617.82 78815.669 436640.548 79179.422 437000.493 79189.724 437010.687 79217.939 437038.607 79222.06 437042.685 79234.824 437055.315 79235.511 437055.995 79235.619 437056.102 79236.547 437057.021 79236.575 437057.049 79236.828 437057.299 79237.362 437057.827 79246.211 437066.584 79241.797 437067.745 79251.998 437076.895 79265.855 437089.325 79266.565 437089.962 79266.976 437090.331 79270.865 437093.819 79271.603 437094.481 79274.551 437097.339 79276.371 437099.103 79276.454 437099.184 79278.22 437100.895 79278.952 437101.605 79280.243 437102.856 79280.778 437103.374 79289.17 437111.509 79290.617 437113.013 79290.944 437113.353 79292.32 437114.782 79332.832 437156.873 79351.883 437176.621 79356.049 437180.939 79400.491 437227.007 79408.615 437235.428 79408.684 437235.5 79408.753 437235.572 79408.996 437235.824 79413.525 437240.518 79467.906 437297.218 79478.002 437307.698 79482.862 437312.743 79495.807 437326.18 79512.718 437343.735 79517.579 437348.781 79553.894 437386.478 79590.122 437423.87 79617.023 437451.635 79662.961 437499.112 79668.524 437504.861 79669.498 437505.868 79673.312 437509.81 79694.22 437531.419 79702.147 437539.612 79648.635 437551.901 79629.654 437556.07 79547.776 437574.055 79535.89 437576.602 79524.584 437579.024 79523.431 437579.271 79522.755 437579.415 79487.252 437587.652 79473.864 437590.758 79473.615 437590.816 79473.6 437590.819 79459.486 437594.094 79441.903 437598.173 79441.059 437598.367 79416.861 437603.916 79407.05 437606.112 79337.962 437621.578 79337.546 437621.671 79303.176 437629.366 79297.172 437630.71 79254.755 437640.298 79245.334 437642.427 79201.654 437651.726 79197.222 437652.732 79129.936 437668.014 79124.273 437669.36 79067.488 437682.867 79053.301 437685.881 78974.076 437702.711 78973.493 437702.843 78972.953 437702.971 78970.692 437703.478 78938.725 437710.486 78893.913 437720.31 78887.703 437719.811 78873.127 437723.034 78840.505 437730.645 78834.421 437732.065 78830.998 437732.864 78779.304 437746.598 78750.924 437752.736 78750.068 437752.288 78741.69 437747.835 78736.637 437747.488 78731.341 437747.781 78719.921 437750.433 78697.235 437755.037 78590.431 437776.997 78543.765 437787.582 78492.964 437794.705 78462.655 437798.899 78428.584 437803.023 78427.042 437803.187 78425.744 437802.294 78422.371 437802.561 78415.542 437802.948 78411.193 437803.746 78407.169 437804.468 78403.345 437805.364 78399.427 437807.362 78398.883 437808.691 78398.516 437810.395 78398.148 437811.873 78398.358 437813.749 78398.465 437815.3 78398.831 437816.58 78387.36 437818.144 78384.284 437811.121 78383.327 437809.875 78381.497 437809.083 78380.344 437808.714 78379.294 437808.895 78377.67 437809.329 78372.896 437810.33 78365.974 437812.018 78360.325 437813.024 78349.952 437815.081 78342.798 437816.509 78342.986 437815.872 78342.075 437816.021 78336.182 437816.822 78334.967 437816.988 78334.479 437817.041 78321.694 437818.436 78315.113 437818.763 78304.99 437817.84 78270.9 437813.259 78253.721 437809.78 78216.678 437811.581 78186.229 437822 78183.165 437823.048 78179.014 437825.793 78169.076 437832.366 78158.416 437839.417 78141.064 437850.893 78138.325 437853.564 78137.757 437854.071 78131.405 437859.743 78128.153 437862.752 78119.24 437871.045 78104.907 437883.386 78082.534 437900.535 78065.629 437914.318 78053.04 437924.255 78040.781 437930.074 78019.773 437940.147 77996.127 437954.058 77954.988 437978.362 77905.672 438008.435 77898.351 438012.899 77887.462 438019.69 77839.637 438049.515 77835.931 438051.976 77827.75 438057.408 77825.679 438058.684 77816.926 438064.075 77760.868 438038.347 77760.09 438040.14 77753.133 438055.51 77752.427 438057.07 77723.757 438109.831 77716.365 438114.749 77716.265 438114.815 77669.442 438146.547 77669.167 438146.735 77636.124 438169.322 77587.173 438183.454 77565.765 438189.3 77540.471 438178.545 77515.89 438230.816 77512.186 438238.736 77507.219 438249.357 77505.316 438248.668 77456.304 438299.476 77440.68 438315.755 77429.484 438327.42 77417.814 438339.864 77412.955 438344.982 77384.727 438370.201 77364.199 438386.021 77358.639 438391.262 77327.755 438420.69 77322.187 438418.774 77276.08 438479.545 77229.19 438542.58 77235.628 438544.496 77201.178 438591.336 77128.886 438688.151 77021.045 438836.207 77003.897 438860.262 76997.078 438869.418 76913.008 438982.296 76899.802 438999.976 76899.786 438999.998 76885.511 439019.108 76848.292 439069.082 76847.151 439070.615 76838.958 439081.615 76816.489 439111.454 76802.386 439130.15 76795.536 439139.453 76773.107 439169.202 76773.1 439169.211 76747.363 439203.348 76709.19 439254.363 76698.887 439268.133 76689.086 439281.182 76643.508 439341.869 76639.919 439346.648 76636.435 439351.286 76584.503 439421.076 76529.845 439494.527 76529.749 439494.498 76487.151 439551.33 76476.158 439565.997 76471.279 439572.517 76463.739 439582.593 76448.094 439603.5 76432.822 439623.908 76361.561 439719.136 76355.643 439717.569 76282.408 439815.51 76288.607 439817.282 76260.575 439854.938 76207.687 439925.887 76201.357 439924.356 76194.427 439933.552 76189.143 439940.512 76177.2 439956.336 76154.812 439986.196 76146.641 439997.103 76143 440002 76136 440000 76099.307 440041.935 76093.141 440048.981 76059 440088 76039 440100 76017.595 440107.37 76014.323 440108.922 75995.52 440113.512 75978.06 440116.742 75955.905 440120.295 75934.195 440122.416 75912.742 440123.319 75899.97 440123.053 75871.583 440122.582 75850.873 440120.048 75842.083 440118.972 75828.785 440116.389 75819.139 440114.516 75797.266 440110.281 75772.713 440103.188 75749.159 440094.752 75730.349 440087.131 75713.069 440080.188 75695.512 440073.2 75679.609 440066.608 75661.63 440059.12 75645.81 440052.209 75627.199 440044.219 75604.629 440034.278 75582.018 440025.636 75576.682 440023.889 75565.79 440020.324 75555.237 440016.902 75553.48 440019.591 75550.19 440024.787 75548.936 440026.959 75546.907 440030.474 75544.642 440034.535 75543.201 440037.312 75539.292 440046.141 75536.967 440051.393 75524.662 440078.206 75521.845 440084.328 75518.224 440092.472 75514.359 440104.666 75513.344 440107.106 75508.283 440118.027 75503.717 440126.794 75493.727 440145.132 75490.464 440150.642 75484.403 440160.757 75472.631 440179.715 75467.174 440189.513 75461.902 440199.306 75459.485 440204.271 75455.692 440215.253 75450.253 440229.405 75445.954 440241.961 75444.653 440245.323 75438.798 440260.182 75432.149 440280.771 75432.143 440280.796 75422.838 440310.084 75417.699 440327.925 75403.974 440375.326 75400.088 440388.059 75389.15 440423.688 75388.514 440425.376 75374.801 440461.039 75363.829 440490.952 75360.433 440501.193 75346.324 440537.223 75342.796 440544.58 75341.454 440547.655 75325.314 440584.634 75322.631 440581.382 75313.299 440569.983 75287.55 440538.532 75118.489 440332.453 75115.071 440328.286 75091.593 440299.646 75033.928 440229.301 75004.874 440193.908 74821.895 439971.009 74756.475 439891.269 74734.317 439864.227 74729.913 439858.853 74711.095 439860.96 74705.91 439861.54 74695.82 439843.61 74565.36 439611.95 74456.08 439418.06 74438.31 439386.52 74437.083 439384.342 74432.949 439377.004 74424.035 439361.183 74421.268 439356.271 74420.14 439354.27 74414.98 439344.75 74411.98 439331.19 74403.22 439318.83 74400.74 439320.36 74376.121 439276.587 74365.16 439257.099 74229 439015 74254.043 439000 74323.3 438955.277 74404.791 438898.417 74449.734 438864.733 74484.237 438838.474 74510.996 438816.389 74557.985 438778.506 74639.2 438714.081 74715.012 438648.295 74788.468 438582.142 74856.597 438516.788 74930.917 438441.202 74974.62 438393.139 74997.5 438367.468 75033.535 438325.424 75062.749 438291.339 75123.399 438213.376 75180.647 438130.392 75235.183 438046.063 75283.861 437960.497 75335.116 437874.697 75384.816 437789.606 75439.991 437704.146 75543.436 437545.351 75548.149 437538.116 75604.112 437454.171 75662.029 437373.036 75720.089 437292.419 75784.091 437215.622 75795.513 437203.263 75822.681 437173.309 75848.17 437144.988 75852.93 437139.887 75865.463 437127.531 75909.017 437083.688 75924.706 437068.27 75960.32 437036.153 75995.707 437004.018 76027.176 436979.281 76062.282 436952.027 76076.718 436940.67 76149.044 436886.135 76236.694 436825.769 76321.418 436776.268 76348.108 436762.563 76360.887 436756.251 76381.795 436745.531 76414.145 436728.945 76502.824 436686.563 76595.41 436643.162 76682.565 436598.726 76772.936 436551.395 76813.413 436529.888 76862.399 436503.21 76949.074 436453.879 77002.929 436421.327 77026.346 436406.914 77094.086 436366.9 77117.168 436352.952 77146.703 436334.156 77218.648 436287.652 77287.671 436242.36 77369.829 436187.153 77457.259 436129.482 77635.927 436009.82 77692.472 435971.268 77789.592 435905.711 77857.957 435861.654 77952.481 435796.88 78009.993 435854.284 78098.862 435942.535 78139.543 435983.394 78146.316 435990.196 78148.795 435992.685 78150.376 435994.273 78151.027 435994.926 78154.36 435998.275 78155.754 435999.675 78156.72 436000.633 78201.037 436044.553"


# Step 1: Convert string to list of (x, y) tuples
coordinates = list(map(float, raw_coordinates.split()))

coords_list = [(coordinates[i], coordinates[i+1]) for i in range(0, len(coordinates), 2)]

# Step 2: Convert UTM to Latitude/Longitude
lat_lon_coords = []
for x, y in coords_list:
    lon, lat = transform(utm_proj, wgs_proj, x, y)
    lat_lon_coords.append((lon, lat))  # Append (longitude, latitude)
    
# Step 3: Create a Polygon from the transformed coordinates
polygon = Polygon(lat_lon_coords)

# Step 4: Create the map centered on the Netherlands
m = folium.Map(location=[52.4, 5.0], zoom_start=8)

# Step 5: Convert the polygon to GeoJSON
geojson = gpd.GeoSeries([polygon]).to_json()

# Step 6: Add the polygon to the map
folium.GeoJson(geojson, name="Polygon").add_to(m)

# Add some styling for the polygon
folium.GeoJson(geojson, name="Polygon", 
               popup="This is my complex polygon",
               style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2}).add_to(m)

# Step 7: Save the map as an HTML file
m.save("../maps/complex_polygon_map.html")


/usr/lib/python3/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/lib/python3/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_36426/1411681864.py:22: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, wgs_proj, x, y)


[(4.271127239851676, 51.90759520500242), (4.271239419788913, 51.90766612392875), (4.27129804251924, 51.90770318497208), (4.271376351916089, 51.907752783810096), (4.2713921481661465, 51.907762794737145), (4.271417102110138, 51.90777859164313), (4.27168213666485, 51.90794645157447), (4.271707405341787, 51.90796245815177), (4.271873054933514, 51.908067368041294), (4.271882712815841, 51.9080735003145), (4.272032308933866, 51.90816824385956), (4.2720571202573145, 51.90818394938231), (4.272486394968508, 51.908455840514264), (4.273326600790531, 51.908970418597946), (4.274275787186835, 51.90954716582024), (4.274786757799009, 51.909860614058324), (4.275028122136453, 51.91000868211391), (4.275089440397372, 51.910046306063926), (4.275964847541056, 51.91057504994245), (4.276806304525844, 51.91108747492893), (4.2769518237420545, 51.91117568346645), (4.276965155413462, 51.911183773662565), (4.2769716281903305, 51.911187691090284), (4.27701550484812, 51.91121604338238), (4.277015833733782, 51.9112162